In [1]:
import pandas as pd
import numpy as np
import requests
import json
import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [6]:
neighborhoods_data = newyork_data['features']


In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [20]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


### Define Fousquare Credentials and Versions 

In [13]:
CLIENT_ID = 'FWRAAQDILXRO0OK05ZKNO4JN1WERLDNU20NAQSRBXEEZIOGB' # your Foursquare ID
CLIENT_SECRET = 'VD4HC2UPTO3VZSYPZZEPNPHO2YS4IJBN14NMZEUAY2PZI2QX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FWRAAQDILXRO0OK05ZKNO4JN1WERLDNU20NAQSRBXEEZIOGB
CLIENT_SECRET:VD4HC2UPTO3VZSYPZZEPNPHO2YS4IJBN14NMZEUAY2PZI2QX


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
limit=200
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [24]:
ny_venues.shape

(10241, 7)

In [62]:
ny_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop
4,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place


In [39]:
x=pd.DataFrame(ny_venues['Neighborhood'].value_counts())

In [42]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,32,32,32,32,32,32
Annadale,9,9,9,9,9,9
Arden Heights,5,5,5,5,5,5
Arlington,8,8,8,8,8,8
Arrochar,20,20,20,20,20,20
Arverne,15,15,15,15,15,15
Astoria,100,100,100,100,100,100
Astoria Heights,14,14,14,14,14,14
Auburndale,21,21,21,21,21,21


In [44]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 431 uniques categories.


## Analyze each Neighborhood

In [83]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

ny_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Arcade,...,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
i=tuple(ny_onehot.columns.values)
i.index('Neighborhood')

266

In [81]:
fixed_columns = [ny_onehot.columns[266]] + list(ny_onehot.columns[:265]) + list(ny_onehot.columns[267:])
ny_onehot = ny_onehot[fixed_columns]

In [76]:
fixed_columns = [ny_onehot.columns[266]] + list(ny_onehot.columns[:265]) + list(ny_onehot.columns[267:])
ny_onehot = ny_onehot[fixed_columns]

In [82]:
ny_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,...,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
ny_onehot.shape

(10241, 431)

In [85]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,...,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allerton,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.031250,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Annadale,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Arden Heights,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Arlington,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.125000,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Arrochar,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Arverne,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Astoria,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.010000,0.000000,0.000000,0.000000
7,Astoria Heights,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Auburndale,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.047619,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Bath Beach,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.00000,0.000000,0.000000,0.000000,0.037037,0.000000


In [86]:
col_name=[]
for col in ny_grouped.columns.values:
    if "Restaurant" in col:
        col_name.append(col)
    

In [87]:
col_name[0:10]

['Afghan Restaurant',
 'African Restaurant',
 'American Restaurant',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Asian Restaurant',
 'Australian Restaurant',
 'Austrian Restaurant',
 'Belgian Restaurant',
 'Brazilian Restaurant']

In [95]:
ny_grouped_1 =  ny_grouped.loc[:,col_name]

In [96]:
print(ny_grouped_1.shape)
ny_grouped_1.head()

(301, 92)


,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Belgian Restaurant,Brazilian Restaurant,...,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,0.0,0.0,0.03125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.12500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
ny_grouped_1['Neighborhood'] = ny_grouped['Neighborhood']

In [98]:
fixed_columns = [ny_grouped_1.columns[-1]] + list(ny_grouped_1.columns[:-1])
ny_grouped_1 = ny_grouped_1[fixed_columns]

ny_grouped_1.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Belgian Restaurant,...,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Allerton,0.0,0.0,0.03125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Annadale,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Arden Heights,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arlington,0.0,0.0,0.12500,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Arrochar,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
num_top_venues = 5

for hood in ny_grouped_1['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped_1[ny_grouped_1['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                  venue  freq
0    Chinese Restaurant  0.06
1   American Restaurant  0.03
2  Fast Food Restaurant  0.03
3    Spanish Restaurant  0.03
4     Afghan Restaurant  0.00


----Annadale----
                     venue  freq
0               Restaurant  0.11
1        Afghan Restaurant  0.00
2  New American Restaurant  0.00
3         Ramen Restaurant  0.00
4    Portuguese Restaurant  0.00


----Arden Heights----
                     venue  freq
0        Afghan Restaurant   0.0
1  New American Restaurant   0.0
2               Restaurant   0.0
3         Ramen Restaurant   0.0
4    Portuguese Restaurant   0.0


----Arlington----
                     venue  freq
0      American Restaurant  0.12
1        Afghan Restaurant  0.00
2  New American Restaurant  0.00
3               Restaurant  0.00
4         Ramen Restaurant  0.00


----Arrochar----
                       venue  freq
0         Italian Restaurant  0.10
1   Mediterranean Restaurant  0.05
2  Middle Eastern Rest

In [100]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the dataframe and display 5 most visited restaurant in each Neighborhood

In [108]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Restaurant'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restaurant'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped_1['Neighborhood']

for ind in np.arange(ny_grouped_1.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped_1.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
0,Allerton,Chinese Restaurant,American Restaurant,Fast Food Restaurant,Spanish Restaurant,Vietnamese Restaurant
1,Annadale,Restaurant,Gluten-free Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant
2,Arden Heights,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
3,Arlington,American Restaurant,Vietnamese Restaurant,Halal Restaurant,English Restaurant,Ethiopian Restaurant
4,Arrochar,Italian Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Greek Restaurant
5,Arverne,Thai Restaurant,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant
6,Astoria,Middle Eastern Restaurant,Greek Restaurant,Seafood Restaurant,Italian Restaurant,Latin American Restaurant
7,Astoria Heights,Italian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Greek Restaurant,English Restaurant
8,Auburndale,Korean Restaurant,American Restaurant,Italian Restaurant,Fast Food Restaurant,Vietnamese Restaurant
9,Bath Beach,Chinese Restaurant,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Restaurant


## Clustering

In [109]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped_1.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 1, 0, 0, 3, 0, 3], dtype=int32)

In [110]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,0.0,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,0.0,Restaurant,Gluten-free Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,2.0,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Chinese Restaurant,Vietnamese Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,0.0,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,0.0,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant


### Examine Clusters

In [111]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
0,Wakefield,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
1,Co-op City,Restaurant,Gluten-free Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant
3,Fieldston,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
4,Riverdale,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
5,Kingsbridge,Latin American Restaurant,Mexican Restaurant,Spanish Restaurant,Fast Food Restaurant,Seafood Restaurant
6,Marble Hill,American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Empanada Restaurant
7,Woodlawn,Italian Restaurant,Indian Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Empanada Restaurant
10,Baychester,American Restaurant,Fast Food Restaurant,Mexican Restaurant,Spanish Restaurant,Vietnamese Restaurant
12,City Island,Seafood Restaurant,Spanish Restaurant,Italian Restaurant,American Restaurant,French Restaurant
15,Morris Heights,Spanish Restaurant,Latin American Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Empanada Restaurant


In [113]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
34,Belmont,Italian Restaurant,Restaurant,Mexican Restaurant,Spanish Restaurant,Japanese Restaurant
36,North Riverdale,Italian Restaurant,Sushi Restaurant,Chinese Restaurant,Mexican Restaurant,Greek Restaurant
39,Edgewater Park,Italian Restaurant,Chinese Restaurant,American Restaurant,Asian Restaurant,Vietnamese Restaurant
66,Carroll Gardens,Italian Restaurant,Thai Restaurant,French Restaurant,Restaurant,Sushi Restaurant
117,Greenwich Village,Italian Restaurant,French Restaurant,Sushi Restaurant,Indian Restaurant,Chinese Restaurant
123,West Village,Italian Restaurant,New American Restaurant,American Restaurant,French Restaurant,Sushi Restaurant
154,Douglaston,Italian Restaurant,Thai Restaurant,Fast Food Restaurant,Chinese Restaurant,Szechuan Restaurant
206,Mariner's Harbor,Italian Restaurant,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant
211,New Dorp,Italian Restaurant,Vietnamese Restaurant,Mexican Restaurant,Dim Sum Restaurant,Chinese Restaurant
213,Great Kills,Italian Restaurant,Japanese Restaurant,Falafel Restaurant,Chinese Restaurant,Spanish Restaurant


In [114]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
2,Eastchester,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Chinese Restaurant,Vietnamese Restaurant
9,Williamsbridge,Caribbean Restaurant,Vietnamese Restaurant,Greek Restaurant,English Restaurant,Ethiopian Restaurant
41,Olinville,Caribbean Restaurant,Fast Food Restaurant,Mexican Restaurant,Vietnamese Restaurant,Greek Restaurant
54,Flatbush,Mexican Restaurant,Caribbean Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant
74,Canarsie,Asian Restaurant,Chinese Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Greek Restaurant
75,Flatlands,Fast Food Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Greek Restaurant,English Restaurant
78,Coney Island,Caribbean Restaurant,Vietnamese Restaurant,Greek Restaurant,English Restaurant,Ethiopian Restaurant
93,Prospect Park South,Caribbean Restaurant,Fast Food Restaurant,Latin American Restaurant,Mexican Restaurant,Chinese Restaurant
165,St. Albans,Caribbean Restaurant,Seafood Restaurant,Chinese Restaurant,Southern / Soul Food Restaurant,Vietnamese Restaurant
168,Cambria Heights,Caribbean Restaurant,Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Empanada Restaurant


In [115]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
8,Norwood,Chinese Restaurant,Spanish Restaurant,Restaurant,American Restaurant,Fast Food Restaurant
11,Pelham Parkway,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Gluten-free Restaurant,Empanada Restaurant
13,Bedford Park,Chinese Restaurant,Mexican Restaurant,Spanish Restaurant,Vietnamese Restaurant,Gluten-free Restaurant
14,University Heights,Fast Food Restaurant,Chinese Restaurant,Spanish Restaurant,African Restaurant,Latin American Restaurant
19,High Bridge,Chinese Restaurant,Seafood Restaurant,Latin American Restaurant,Vietnamese Restaurant,Gluten-free Restaurant
22,Port Morris,Latin American Restaurant,Restaurant,Peruvian Restaurant,Chinese Restaurant,Spanish Restaurant
26,Soundview,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Greek Restaurant,English Restaurant
28,Throgs Neck,American Restaurant,Italian Restaurant,Asian Restaurant,Chinese Restaurant,Vietnamese Restaurant
37,Pelham Bay,Italian Restaurant,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Thai Restaurant
42,Pelham Gardens,Italian Restaurant,American Restaurant,Chinese Restaurant,Spanish Restaurant,Vietnamese Restaurant


In [116]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
181,Floral Park,Indian Restaurant,Vietnamese Restaurant,Greek Restaurant,Empanada Restaurant,English Restaurant
